In [1]:
import pandas as pd
import datetime as dt
from pathlib import Path
from tqdm import tqdm
import time
import os
import gc

In [2]:
def humanbytes(B):
    """Return the given bytes as a human friendly KB, MB, GB, or TB string."""
    B = float(B)
    KB = float(1024)
    MB = float(KB ** 2) # 1,048,576
    GB = float(KB ** 3) # 1,073,741,824
    TB = float(KB ** 4) # 1,099,511,627,776

    if B < KB:
        return '{0} {1}'.format(B,'Bytes' if 0 == B > 1 else 'Byte')
    elif KB <= B < MB:
        return '{0:.2f} KB'.format(B / KB)
    elif MB <= B < GB:
        return '{0:.2f} MB'.format(B / MB)
    elif GB <= B < TB:
        return '{0:.2f} GB'.format(B / GB)
    elif TB <= B:
        return '{0:.2f} TB'.format(B / TB)

In [3]:
filepath = '/home/jovyan/datasets/Ftplogs/Output/20220318_SESV400_FtpLogs.csv'
txt = Path(filepath).resolve()
length = sum(1 for row in open(txt, 'r'))
print(length)
filesize = os.path.getsize(filepath)
print(f'%10s' % humanbytes(filesize))

9233768
   1.18 GB


In [4]:
dsz = length
chunksize = 5000
df = pd.DataFrame()
order = [
    "date","time","c-ip","cs-username","s-ip",
    "s-port","cs-method","cs-uri-stem","sc-status","sc-win32-status",
    "sc-substatus","time-taken","x-session","x-fullpath"
]
with tqdm(total=length, desc="chunks read: ") as bar:
    for i, chunk in enumerate(pd.read_csv(txt, chunksize=chunksize, low_memory=False, index_col=0)):
        df = pd.concat([df,chunk])
        bar.update(min(dsz, chunksize))
        dsz -= chunksize
        gc.collect()

        

chunks read:  98%|█████████▊| 9085000/9233768 [1:04:06<01:02, 2361.64it/s]


KeyboardInterrupt: 

In [ ]:
print(df.info())
print(len(df))

In [15]:
import dask.dataframe as ddf
import dask.multiprocessing
import pickle

In [2]:
%%time
#filepath = '/home/jovyan/datasets/Ftplogs/Output/20220318_SESV400_FtpLogs.csv'
filepath = '/home/jovyan/datasets/Ftplogs/Output/20220318_RGSV920_FtpLogs.csv'
picklepath = '/home/jovyan/datasets/Ftplogs/Output/20220318_RGSV920_FtpLogs.csv.pickle'
df_dask = ddf.read_csv(filepath)
df_dask = df_dask.compute()
#display(df_dask.tail())
print(df_dask.info())
df_dask.to_pickle(picklepath)

NameError: name 'ddf' is not defined

In [1]:
%%time
import pickle
with open(picklepath, mode='rb') as fp:
    df_pickle = pickle.load(fp)
display(df_pickle.info())

NameError: name 'picklepath' is not defined